In [1]:
import os
os.getcwd()
os.chdir("../")
os.getcwd()

'c:\\Users\\Marina\\Desktop\\ML Operations\\0 - KrishNaik Course\\18 - Getting Started With Your First End To End Data Science Project With\\my_project'

## Defining the Schema

In [2]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


Vamos jogar estas features para o `my_project/schema.yaml`

## Creating the Dataclass

**Todos os elementos do config.yaml devem estar nesta classe**

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig():
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str

    schema: dict 


## Update the Config Manager

In [7]:
from src.datascience.constants import CONFIG_FILEPATH, PARAMS_FILEPATH, SCHEMA_FILEPATH
from src.datascience.utils.commons import read_yaml, create_directories

class ConfigurationManager():
    def __init__(self, # recebe os parâmetros que seram usados para instanciar as propriedades p/leitura
                 config_filepath = CONFIG_FILEPATH,
                 params_filepath = PARAMS_FILEPATH,
                 schema_filepath = SCHEMA_FILEPATH,):
        
        # instância as propriedades com base nas leituras dos parâmetros passados acima
        self.config = read_yaml(config_filepath) # lê as configurações
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root]) # cria o /artifacts

    def get_data_validation_config(self) -> DataValidationConfig:
        
        config = self.config.data_validation # change the "section" of the config file
        schema = self.schema.FEATURE_COLUMNS
        
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            schema = schema)
        
        return data_validation_config

## Update the Components

In [ ]:
import urllib.request as request
import pandas as pd
from src.datascience.utils import logger
import zipfile


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        # config should be an instance of DataValidationConfig
        self.config = config

    def validate_column_names(self) -> bool:
        """Validate if all columns match the schema names."""
        try:
            validation_status_names = True
            log_messages = []

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            schema_columns = set(self.config.schema.keys())

            for col in all_cols:
                if col not in schema_columns:
                    validation_status_names = False
                    log_messages.append(f"Name Validation Status for {col}: False")
                else:
                    log_messages.append(f"Name Validation Status for {col}: True")

            # Write all messages to the status file at once
            with open(self.config.STATUS_FILE, "a") as f:
                f.write("\n".join(log_messages) + "\n")

            return validation_status_names

        except Exception as e:
            logger.exception(f"Error during column name validation: {e}")
            raise e

    def validate_column_types(self) -> bool:
        """Validate if all column types match the schema types."""
        try:
            validation_status_types = True
            log_messages = []

            data = pd.read_csv(self.config.unzip_data_dir)
            column_types = data.dtypes.to_dict()
            schema_types = self.config.schema

            for col, dtype in column_types.items():
                if col not in schema_types or schema_types[col] != str(dtype):
                    validation_status_types = False
                    log_messages.append(f"Dtype Validation Status for {col}: False")
                else:
                    log_messages.append(f"Dtype Validation Status for {col}: True")

            # Write all messages to the status file at once
            with open(self.config.STATUS_FILE, "a") as f:
                f.write("\n".join(log_messages) + "\n")

            return validation_status_types

        except Exception as e:
            logger.exception(f"Error during column type validation: {e}")
            raise e


## Testing

In [21]:
from src.datascience.utils import logger

try:
    # Get the variables from needed for the process
    data_validation_config = ConfigurationManager().get_data_validation_config()
    
    # Instantiate the object for performing the Data Ingestion
    data_validation_object = DataValidation(data_validation_config)

    # Download and extract
    validate_column_names_status = data_validation_object.validate_column_names()
    validate_column_type_status = data_validation_object.validate_column_types()
    print(f"validate_column_names_status: {validate_column_names_status} \nvalidate_column_type_status: {validate_column_type_status}")

except Exception as e:
    logger.error(f"Error during data_ingestion. Error {e}")
    raise(e)

[2024-11-04 11:05:12,245: INFO: commons: yaml file: config\config.yaml loaded successfully ]
[2024-11-04 11:05:12,253: INFO: commons: yaml file: params.yaml loaded successfully ]
[2024-11-04 11:05:12,257: INFO: commons: yaml file: schema.yaml loaded successfully ]
[2024-11-04 11:05:12,259: INFO: commons: created directory at: artifacts ]
[2024-11-04 11:05:12,261: INFO: commons: created directory at: artifacts/data_validation ]
validate_column_names_status: True 
validate_column_type_status: True


## Agora iremos converter para código modular

Começamos do passo 4 em diante(do workflow em notes.ipynb)

O código inteiro da seção `Creating the Dataclass` vai para `src/entity/config_entity.py`

O código inteiro da seção `Update the Config Manager` vai para `src/config/configuration.py`

O código inteiro da seção `Update the Components` vai para `src/components/nome_da_step.py`

O código inteiro da seção `Testing` vai para `src/pipeline/nome_da_step_pipeline.py`, com implementação OOP, ver exemplo 

O código anterior, vai para `main.py`, podendo chamar diretamente a classe, só ver o exemplo